In [2]:
import pandas as pd
import h2o

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 18.0.2.1+1-1, mixed mode, sharing)
  Starting server from C:\Users\PV\AppData\Local\Programs\Python\Python311\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\PV\AppData\Local\Temp\tmp5s_9r6j4
  JVM stdout: C:\Users\PV\AppData\Local\Temp\tmp5s_9r6j4\h2o_PV_started_from_python.out
  JVM stderr: C:\Users\PV\AppData\Local\Temp\tmp5s_9r6j4\h2o_PV_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Asia/Ho_Chi_Minh
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.3
H2O_cluster_version_age:,2 days
H2O_cluster_name:,H2O_from_python_PV_q3r1m7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.947 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [4]:
from h2o.automl import H2OAutoML

In [5]:
churn_df = h2o.import_file('https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/WA_Fn-UseC_-Telco-Customer-Churn.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
churn_df.types

{'customerID': 'string',
 'gender': 'enum',
 'SeniorCitizen': 'int',
 'Partner': 'enum',
 'Dependents': 'enum',
 'tenure': 'int',
 'PhoneService': 'enum',
 'MultipleLines': 'enum',
 'InternetService': 'enum',
 'OnlineSecurity': 'enum',
 'OnlineBackup': 'enum',
 'DeviceProtection': 'enum',
 'TechSupport': 'enum',
 'StreamingTV': 'enum',
 'StreamingMovies': 'enum',
 'Contract': 'enum',
 'PaperlessBilling': 'enum',
 'PaymentMethod': 'enum',
 'MonthlyCharges': 'real',
 'TotalCharges': 'real',
 'Churn': 'enum'}

In [7]:
churn_df.describe()

Rows:7043
Cols:21

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
type,string,enum,int,enum,enum,int,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,real,real,enum
mins,NaN,,0.0,,,0.0,,,,,,,,,,,,,18.25,18.8,
mean,NaN,,0.1621468124378816,,,32.37114865824223,,,,,,,,,,,,,64.76169246059919,2283.3004408418656,
maxs,NaN,,1.0,,,72.0,,,,,,,,,,,,,118.75,8684.8,
sigma,NaN,,0.3686116056100131,,,24.55948102309446,,,,,,,,,,,,,30.090047097678493,2266.771361883145,
zeros,0,,5901,,,11,,,,,,,,,,,,,0,0,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0
0,7590-VHVEG,Female,0.0,Yes,No,1.0,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0.0,No,No,34.0,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0.0,No,No,2.0,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


In [8]:
churn_train,churn_test,churn_valid = churn_df.split_frame(ratios=[.7, .15])

In [9]:
y = "Churn"
x = churn_df.columns
x.remove(y)
x.remove("customerID")

In [10]:
aml = H2OAutoML(max_models = 10, seed = 10, exclude_algos = ["StackedEnsemble", "DeepLearning"], verbosity="info", nfolds=0)

In [3]:
!nvidia-smi

Sat Nov 26 11:55:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 526.98       Driver Version: 526.98       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   44C    P0    N/A /  N/A |      0MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
aml.train(x = x, y = y, training_frame = churn_train, validation_frame=churn_valid)

AutoML progress: |
02:05:58.335: Project: AutoML_1_20221126_20558
02:05:58.337: Cross-validation disabled by user: no fold column nor nfolds > 1.
02:05:58.337: Setting stopping tolerance adaptively based on the training frame: 0.014230640818761332
02:05:58.337: Build control seed: 10
02:05:58.337: training frame: Frame key: AutoML_1_20221126_20558_training_py_2_sid_b222    cols: 21    rows: 4938  chunks: 32    size: 568011  checksum: -4784000253246723441
02:05:58.337: validation frame: Frame key: py_4_sid_b222    cols: 21    rows: 1088  chunks: 32    size: 441322  checksum: -6342904108662596759
02:05:58.338: leaderboard frame: Frame key: py_4_sid_b222    cols: 21    rows: 1088  chunks: 32    size: 441322  checksum: -6342904108662596759
02:05:58.338: blending frame: NULL
02:05:58.338: response column: Churn
02:05:58.338: fold column: null
02:05:58.338: weights column: null
02:05:58.355: AutoML: XGBoost is not available; skipping it.
02:05:58.360: Loading execution steps: [{XGBoost : [de

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_grid_1_AutoML_1_20221126_20558_model_1


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    55                 55                          8653                   3            3            3             6             8             7.89091

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.12597277010024943
RMSE: 0.3549264291374333
LogLoss: 0.3921620737228604
Mean Per-Class Error: 0.22208187776077604
AUC: 0.8655279164239532
AUCPR: 0.6934896371648036
Gini: 0.7310558328479064

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3673562951406781
       No    Yes    Error    Rate
-----  ----  -----  -------  --------------
No     3082  564    0.1547   (564.0/3646.0)
Yes    374   918    0.2895   (374.0/1292.0)
Total  3456  1482   0.19     (938.0/4938.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.367356     0.66186   201
max f2                       0.193149     0.758845  281
max f0point5                 0.492852     0.664719  151
max accuracy                 0.492852     0.821385  151
max precision                0.893046     1         0
max recall                   0.0219098    1         396
max specificity              0.893046     1         0
max absolute_mcc             0.367356     0.533059  201
max min_per_class_accuracy   0.297016     0.785604  232
max mean_per_class_accuracy  0.292872     0.787124  234
max tns                      0.893046     3646      0
max fns                      0.893046     1289      0
max fps                      0.0193343    3646      399
max tps                      0.0219098    1292      396
max tnr                      0.893046     1         0
max fnr                      0.893046     0.997678  0
max fpr                      0.0193343    1         399
max tpr                      0.0219098    1         396

Gains/Lift Table: Avg response rate: 26.16 %, avg score: 26.14 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0101256                   0.840859           3.51622    3.51622            0.92             0.86199    0.92                        0.86199             0.0356037       0.0356037                  251.622   251.622            0.0345066
2        0.0200486                   0.810829           3.27598    3.39732            0.857143         0.82596    0.888889                    0.844157            0.0325077       0.0681115                  227.598   239.732            0.0650945
3        0.0301742                   0.778926           3.36334    3.38592            0.88             0.794069   0.885906                    0.827349            0.0340557       0.102167                   236.334   238.592            0.0975045
4        0.0400972                   0.75849            3.27598    3.35871            0.857143         0.767219   0.878788                    0.812468            0.0325077       0.134675                   227.598   235.871            0.128092
5        0.0500203                   0.73653            3.35398    3.35777            0.877551         0.746564   0.878543 

In [13]:
lb = aml.leaderboard

In [14]:
lb.head()

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
GBM_grid_1_AutoML_1_20221126_20558_model_1,0.855845,0.403985,0.697309,0.21284,0.360013,0.129609
GBM_1_AutoML_1_20221126_20558,0.855403,0.403777,0.694553,0.226496,0.359924,0.129546
GLM_1_AutoML_1_20221126_20558,0.852077,0.411903,0.689534,0.219355,0.36221,0.131196
GBM_5_AutoML_1_20221126_20558,0.851934,0.409238,0.681231,0.224766,0.363964,0.13247
GBM_2_AutoML_1_20221126_20558,0.851391,0.409976,0.68871,0.214349,0.362823,0.131641
GBM_3_AutoML_1_20221126_20558,0.84227,0.421618,0.668298,0.245123,0.368155,0.135538
GBM_4_AutoML_1_20221126_20558,0.842229,0.426892,0.669678,0.244202,0.370026,0.136919
XRT_1_AutoML_1_20221126_20558,0.841209,0.42034,0.669962,0.236656,0.366509,0.134329
DRF_1_AutoML_1_20221126_20558,0.829703,0.495068,0.631658,0.24111,0.37508,0.140685
GBM_grid_1_AutoML_1_20221126_20558_model_2,0.828674,0.455557,0.645772,0.247957,0.380735,0.144959


In [15]:
churn_pred=aml.leader.predict(churn_test)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [16]:
churn_pred.head()

predict,No,Yes
Yes,0.196427,0.803573
No,0.832228,0.167772
No,0.973495,0.0265052
Yes,0.630127,0.369873
Yes,0.221321,0.778679
Yes,0.534179,0.465821
No,0.958619,0.0413812
No,0.884783,0.115217
No,0.9682,0.0318003
Yes,0.546159,0.453841


In [17]:
aml.leader.model_performance(churn_test)

ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.14573259352900705
RMSE: 0.3817493857611392
LogLoss: 0.4432883220955206
Mean Per-Class Error: 0.24410579561042522
AUC: 0.8321949778997104
AUCPR: 0.6599298022725674
Gini: 0.6643899557994208

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.2904771427755448
       No    Yes    Error    Rate
-----  ----  -----  -------  --------------
No     568   161    0.2209   (161.0/729.0)
Yes    77    211    0.2674   (77.0/288.0)
Total  645   372    0.234    (238.0/1017.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.290477     0.639394  210
max f2                       0.0728545    0.75431   341
max f0point5                 0.525109     0.627323  116
max accuracy                 0.525109     0.788594  116
max precision                0.892511     1         0
max recall                   0.0255518    1         389
max specificity              0.892511     1         0
max absolute_mcc             0.290477     0.478739  210
max min_per_class_accuracy   0.275668     0.754458  220
max mean_per_class_accuracy  0.256611     0.759624  230
max tns                      0.892511     729       0
max fns                      0.892511     286       0
max fps                      0.0194071    729       399
max tps                      0.0255518    288       389
max tnr                      0.892511     1         0
max fnr                      0.892511     0.993056  0
max fpr                      0.0194071    1         399
max tpr                      0.0255518    1         389

Gains/Lift Table: Avg response rate: 28.32 %, avg score: 26.25 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0108161                   0.841811           3.21023    3.21023            0.909091         0.866548   0.909091                    0.866548            0.0347222       0.0347222                  221.023   221.023            0.0333505
2        0.0216323                   0.813394           2.56818    2.8892             0.727273         0.82209    0.818182                    0.844319            0.0277778       0.0625                     156.818   188.92             0.057013
3        0.0304818                   0.788142           3.53125    3.0756             1                0.800727   0.870968                    0.831664            0.03125         0.09375                    253.125   207.56             0.088263
4        0.0403147                   0.76891            3.53125    3.18674            1                0.778129   0.902439                    0.818606            0.0347222       0.128472                   253.125   218.674            0.122985
5        0.0501475                   0.741846           3.17813    3.18505            0.9              0.749619   0.901961                    0.805079            0.03125         0.159722                   217.812   218.505            0.152864
6        0.100295                    0.652268           2.21569    2.70037            0.627451         0.692854   0.764706                    0.748967            0.111111        0.270833                   121.569   170.037            0.237912
7        0.150442                    0.568573           2.07721    2.49265            0.588235         0.606358   0.705882                    0.70143             0.104167        0.375                      107.721   149.265            0.313272
8        0.20059  

In [18]:
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])
#se = h2o.get_model([mid for mid in model_ids if "StackedEnsemble_AllModels" in mid][0])
#metalearner = h2o.get_model(se.metalearner()['name'])

In [19]:
model_ids

['GBM_grid_1_AutoML_1_20221126_20558_model_1',
 'GBM_1_AutoML_1_20221126_20558',
 'GLM_1_AutoML_1_20221126_20558',
 'GBM_5_AutoML_1_20221126_20558',
 'GBM_2_AutoML_1_20221126_20558',
 'GBM_3_AutoML_1_20221126_20558',
 'GBM_4_AutoML_1_20221126_20558',
 'XRT_1_AutoML_1_20221126_20558',
 'DRF_1_AutoML_1_20221126_20558',
 'GBM_grid_1_AutoML_1_20221126_20558_model_2']